In [2]:
%matplotlib inline
import tensorflow as tf
import tensorflow_probability as tfp
import tensorflow_probability.python.distributions as tfd

import numpy as np
import matplotlib.pyplot as plt

In [444]:
from IPython.display import display, HTML
import numpy as np    

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

# \[Tensorflow Distribution의 역전파\]
---

Tensorflow에서 지원하는 Distribution은 역전파가 가능합니다! 달리 말해, Distribution의 Parameter들은 **학습 가능합니다.**

## 1. 정규분포로 학습시키기 
---

### (1) 학습시킬 정규분포 정의하기

우리는 정규분포의 모수(Parameter)를 Variable로 지정할 수 있습니다. <br>
Tensorflow에서의 Variable은 Training 과정 중에 값이 Update될 수 있는 것을 의미합니다.

In [483]:
graph = tf.Graph()
with graph.as_default():
    mean = tf.Variable(0.5, name='mean')
    std_ = tf.Variable(1.3, name='std_')

    # Standard deviation은 항상 양수여야 하므로, 
    # Softplus을 통해 그 값이 [0,+inf) 이도록 합니다.
    std = tf.nn.softplus(std_,name='std')
    P = tfd.Normal(mean, std,name='P')

In [484]:
show_graph(graph)

### (2) Loss 함수 정의하기

두 분포의 차이를 대표하는 함수는 KullBack-Leibler divergence입니다.

In [485]:
with graph.as_default():
    std_normal = tfd.Normal(0.,1.,name='standard_normal') 
    loss = P.kl_divergence(std_normal)

In [486]:
show_graph(graph)

### (3) optimizer 정의하기

경사하강법(Gradient Decsent Method)을 통해, 필요한 standard deviation과 mean을 찾아보도록 하겠습니다.

In [492]:
with graph.as_default():
    train_op = (tf.train
                .MomentumOptimizer(learning_rate=.5,momentum=.9)
                .minimize(loss))

### (3) 모델 학습시키기

In [493]:
with tf.Session(graph=graph) as sess:
    # Mean, STD_ 변수 초기화 하기
    sess.run(tf.global_variables_initializer())
    
    for i in range(200):
        sess.run(train_op)
        std_value, mean_value = sess.run([std,mean])
        if i %10 ==0:
            print(f"{i}번째 std : {std_value:.3f} | mean : {mean_value:.3f}")

0번째 std : 1.277 | mean : 0.250
10번째 std : 1.110 | mean : -0.165
20번째 std : 1.022 | mean : -0.171
30번째 std : 0.985 | mean : -0.029
40번째 std : 0.974 | mean : 0.045
50번째 std : 0.976 | mean : 0.033
60번째 std : 0.982 | mean : 0.001
70번째 std : 0.988 | mean : -0.011
80번째 std : 0.993 | mean : -0.006
90번째 std : 0.996 | mean : 0.001
100번째 std : 0.998 | mean : 0.002
110번째 std : 0.999 | mean : 0.001
120번째 std : 1.000 | mean : -0.000
130번째 std : 1.000 | mean : -0.001
140번째 std : 1.000 | mean : -0.000
150번째 std : 1.000 | mean : 0.000
160번째 std : 1.000 | mean : 0.000
170번째 std : 1.000 | mean : 0.000
180번째 std : 1.000 | mean : -0.000
190번째 std : 1.000 | mean : -0.000
